In [39]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from pandas.tseries.offsets import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cvxpy as cp

In [40]:
# ---- STEP 1: Load and Merge Data ----
# Load relevant columns from hackathon_sample_v2.csv and output.csv
hackathon_sample_v2_path = "/Users/isaiah/Desktop/Career/Clubs : Groups/Quant Hackathon/McGill-FIAM Asset Management Hackathon/Quant-Hackathon/hackathon_sample_v2.csv"
output_path = "/Users/isaiah/Desktop/Career/Clubs : Groups/Quant Hackathon/McGill-FIAM Asset Management Hackathon/Quant-Hackathon/output.csv"

hackathon_sample_v2 = pd.read_csv(hackathon_sample_v2_path, usecols=['permno', 'date', 'market_equity', 'be_me', 'ret_12_1', 'ivol_capm_21d', 'stock_exret'])
output = pd.read_csv(output_path, usecols=['permno', 'date', 'en'])

# Convert 'date' in hackathon_sample_v2 from YYYYMMDD integer format to datetime
hackathon_sample_v2['date'] = pd.to_datetime(hackathon_sample_v2['date'], format='%Y%m%d')

# Convert 'date' in output to datetime if needed
output['date'] = pd.to_datetime(output['date'])
# Now merge the data on 'permno' and 'date'
pred = pd.merge(hackathon_sample_v2, output, on=['permno', 'date'], how='inner')

null_counts = pred.isnull().sum()
print("Number of null values per column:")
print(null_counts)

# Drop rows that have any null values
pred_cleaned = pred.dropna(axis=0)

# Display the number of rows before and after dropping rows with null values
print("\nNumber of rows before dropping rows with null values:", len(pred))
print("Number of rows after dropping rows with null values:", len(pred_cleaned))

pred = pred_cleaned

Number of null values per column:
date                0
permno              0
stock_exret         0
market_equity       0
ret_12_1          353
be_me            4808
ivol_capm_21d       1
en                  0
dtype: int64

Number of rows before dropping rows with null values: 158980
Number of rows after dropping rows with null values: 153848


In [41]:
# ---- STEP 2: Multi-Signal Ensemble with Random Forest ----
# Include multiple signals (value, momentum, risk) alongside ElasticNet predictions
pred['value_signal'] = pred['be_me']  # Book-to-market ratio (value factor)
pred['momentum_signal'] = pred['ret_12_1']  # 12-month return (momentum factor)
pred['risk_signal'] = 1 / pred['ivol_capm_21d']  # Inverse volatility (risk factor)
pred['en_signal'] = pred['en']  # ElasticNet-predicted return

# Define the features (signals) and target (actual returns)
features = pred[['value_signal', 'momentum_signal', 'risk_signal', 'en_signal']]
target = pred['stock_exret']

# Train the Random Forest model to generate predicted returns from signals
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Generate predicted returns based on the multi-signal ensemble (including EN predictions)
pred['predicted_return'] = rf_model.predict(features)


In [42]:
# ---- STEP 3: Black-Litterman Optimization (Monthly Rebalancing) ----
# Group the data by 'year' and 'month'
monthly_results = []

for (year, month), monthly_data in pred_cleaned.groupby([pred_cleaned['date'].dt.year, pred_cleaned['date'].dt.month]):
    
    # Define market weights and equilibrium return
    monthly_data['market_weight'] = monthly_data['market_equity'] / monthly_data['market_equity'].sum()
    market_equilibrium_return = np.dot(monthly_data['market_weight'], monthly_data['stock_exret'])
    views = monthly_data['predicted_return']

    tau = 0.05
    bl_adjusted_returns = (1 - tau) * market_equilibrium_return + tau * views
    bl_adjusted_returns = np.array(bl_adjusted_returns)

    # Create covariance matrix based on asset returns
    pivoted_returns = monthly_data.pivot(index='date', columns='permno', values='stock_exret')
    cov_matrix = pivoted_returns.cov().values

    # Ensure the covariance matrix is symmetric
    cov_matrix = (cov_matrix + cov_matrix.T) / 2

    # Number of assets
    n_assets = len(bl_adjusted_returns)
    
    # Optimization for portfolio weights
    weights = cp.Variable(n_assets)
    portfolio_return = cp.matmul(bl_adjusted_returns, weights)
    portfolio_risk = cp.quad_form(weights, cov_matrix)
    objective = cp.Maximize(portfolio_return - portfolio_risk)
    constraints = [cp.sum(weights) == 1]
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Get the optimized portfolio weights
    optimal_weights = weights.value
    monthly_data['optimal_weight'] = optimal_weights

    # Calculate portfolio return
    monthly_data['portfolio_return'] = monthly_data['stock_exret'] * monthly_data['optimal_weight']
    
    # Store the monthly results
    monthly_results.append(monthly_data)

# Combine all monthly data back into one dataframe
final_portfolio = pd.concat(monthly_results)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/isaiah/opt/anaconda3/envs/ELEC292/lib/python3.9/site-packages/pandas/core/frame.py:10869: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/Users/isaiah/opt/anaconda3/envs/ELEC292/lib/python3.9/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/isaiah/opt/anaconda3/envs/ELEC292/lib/python3.9/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: Quadratic form matrices must be symmetric/Hermitian.

In [ ]:
# ---- STEP 4: Calculate Performance Metrics Based on Rebalanced Portfolio ----

# 1. Sharpe Ratio (Annualized)
sharpe_ratio = final_portfolio['portfolio_return'].mean() / final_portfolio['portfolio_return'].std() * np.sqrt(12)
print("Sharpe Ratio:", sharpe_ratio)

# 2. Annualized Return
annualized_return = final_portfolio['portfolio_return'].mean() * 12
print("Annualized Return:", annualized_return)

# 3. Annualized Standard Deviation
annualized_std_dev = final_portfolio['portfolio_return'].std() * np.sqrt(12)
print("Annualized Standard Deviation:", annualized_std_dev)

In [ ]:
# ---- STEP 5: CAPM Alpha and Information Ratio ----
mkt_path = "/Users/isaiah/Desktop/Career/Clubs : Groups/Quant Hackathon/McGill-FIAM Asset Management Hackathon/mkt_ind.csv"
mkt = pd.read_csv(mkt_path)

# Create 'mkt_rf' by subtracting the risk-free rate from market return
mkt['mkt_rf'] = mkt['sp_ret'] - mkt['rf']
final_portfolio = final_portfolio.merge(mkt, how="inner", on=["year", "month"])

# Perform CAPM regression (Newey-West standard errors)
nw_ols = sm.ols(formula="portfolio_return ~ mkt_rf", data=final_portfolio).fit(cov_type="HAC", cov_kwds={"maxlags": 3}, use_t=True)
print(nw_ols.summary())

# CAPM Alpha, Information Ratio
alpha = nw_ols.params["Intercept"]
t_stat = nw_ols.tvalues["Intercept"]
info_ratio = alpha / np.sqrt(nw_ols.mse_resid) * np.sqrt(12)

print("CAPM Alpha:", alpha)
print("t-statistic:", t_stat)
print("Information Ratio:", info_ratio)


In [ ]:
# ---- STEP 6: Maximum Drawdown ----
final_portfolio["log_portfolio_return"] = np.log(final_portfolio["portfolio_return"] + 1)
final_portfolio["cumsum_log_portfolio_return"] = final_portfolio["log_portfolio_return"].cumsum(axis=0)
rolling_peak = final_portfolio["cumsum_log_portfolio_return"].cummax()
drawdown = rolling_peak - final_portfolio["cumsum_log_portfolio_return"]
max_drawdown = drawdown.max()
print("Maximum Drawdown:", max_drawdown)

In [ ]:
# ---- STEP 7: Maximum Drawdown ----
final_portfolio["log_portfolio_return"] = np.log(final_portfolio["portfolio_return"] + 1)
final_portfolio["cumsum_log_portfolio_return"] = final_portfolio["log_portfolio_return"].cumsum(axis=0)
rolling_peak = final_portfolio["cumsum_log_portfolio_return"].cummax()
drawdown = rolling_peak - final_portfolio["cumsum_log_portfolio_return"]
max_drawdown = drawdown.max()
print("Maximum Drawdown:", max_drawdown)


In [ ]:
# ---- STEP 7: Turnover Calculation ----
def turnover_count(df):
    stock_counts = df.groupby(["year", "month"])["permno"].count().reset_index(name="count")
    stock_counts["count_next"] = stock_counts["count"].shift(-1)
    stock_counts["turnover"] = abs(stock_counts["count"] - stock_counts["count_next"]) / stock_counts["count"]
    return stock_counts["turnover"].mean()

long_portfolio = final_portfolio.nlargest(50, 'optimal_weight')
short_portfolio = final_portfolio.nsmallest(50, 'optimal_weight')

long_turnover = turnover_count(long_portfolio)
short_turnover = turnover_count(short_portfolio)
print("Long Portfolio Turnover:", long_turnover)
print("Short Portfolio Turnover:", short_turnover)
